# Simple Gates Interpretation

## NOR gate with neural network
- NOR gate is a universal gate => a gate which can implement any boolean function
- NOR truth table:
| INPUT1 | INPUT2 | OUTPUT |
| 0 | 0 | 1 |
| 0 | 1 | 0 |
| 1 | 0 | 0 |
| 1 | 1 | 0 |

![](../images/single_neuron_with_sigmoid.png)
- NOR can be represented by a neuron with 2 inputs (W0, W1) and bias (b) with sigmoid as an activation function
- the needed weights can be derived by looking at individual cases
1) case (0, 0)
sig(W0 + W1 + b) = y
sig(0 + 0 + b) = 1
=> b has to be positive, eg. 1
2) case (0, 1)
sig(W1 - 1) = 0 => set W1 to -2
3) case (1, 0)
4) case (1, 1)

## AND gate
- AND truth table:
| INPUT1 | INPUT2 | OUTPUT |
| 0 | 0 | 0 |
| 0 | 1 | 0 |
| 1 | 0 | 0 |
| 1 | 1 | 1 |

1. case (0,0)
b has to be negative, eg. -3
2. case (0,1)
sig(b+0+W2*1) should be less than 0
W0+W2<0 eg. W2 = 2
3. case (1,0)
W1 = 2
4. (1, 1)
-3,2,2

## OR gate
- AND truth table:
| INPUT1 | INPUT2 | OUTPUT |
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 1 |
1. case (0,0)
b has to be negative value, eg.-1
2. case (0,1)
sig(b+0+W2*1)
sig(-1+W2)
then W2 has to be bigger than b, eg.2
3. case (1,0)
same reasoning W2 has to be bigger than b, eg.2
4. case (1,1)
works by chosen values

![](../images/xor_and_or_nand.png)

## Minimum weight set
- construct linear program that solves the constraints from individual gates

In [22]:
import gurobipy as g
model = g.Model()

bias = model.addVar(vtype=g.GRB.CONTINUOUS, lb=-10, ub=10, name="bias")
weight1 = model.addVar(vtype=g.GRB.CONTINUOUS, lb=-10, ub=10, name="weight1")
weight2 = model.addVar(vtype=g.GRB.CONTINUOUS, lb=-10, ub=10, name="weight2")

# model.setObjective(bias + weight1 + weight2, sense=g.GRB.MAXIMIZE)

# AND constraints
model.addConstr(bias + 1*weight1 + 1*weight2 >= 0.1)
model.addConstr(bias + 0*weight1 + 1*weight2 <= -0.1)
model.addConstr(bias + 1*weight1 + 0*weight2 <= -0.1)
model.addConstr(bias + 0*weight1 + 0*weight2 <= -0.1)
# OR constraints
model.addConstr(bias + 1*weight1 + 1*weight2 >= 0.1)
model.addConstr(bias + 0*weight1 + 1*weight2 >= 0.1)
model.addConstr(bias + 1*weight1 + 0*weight2 >= 0.1)
model.addConstr(bias + 0*weight1 + 0*weight2 <= -0.1)
# NAND constraints


model.optimize()

print(bias.x)
print(weight1.x)
print(weight2.x)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 8 rows, 3 columns and 16 nonzeros
Model fingerprint: 0xd7daf6df
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e-01, 1e-01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds
Infeasible model


AttributeError: Unable to retrieve attribute 'x'

### min weight set for AND and NOR
| GATE | BIAS | WEIGHT1 | WEIGHT2 |
| AND  |   -6  |    4    |    4    |
| NOR  |   4  |    -6    |    -6    |


## Possible activation functions
- sigmoid
- threshold function
- hyperbolic tanget

In [ ]:
# global imports
import torch

In [42]:
import math

def stable_sigmoid(x):

    if x >= 0:
        z = math.exp(-x)
        sig = 1 / (1 + z)
        return sig
    else:
        z = math.exp(x)
        sig = z / (1 + z)
        return sig

stable_sigmoid(3)

0.9525741268224334

# Algorithms

## training ternary neural networks by rectified L2 regularization
use it for.:
### min weight set for AND and NOR
| GATE | BIAS | WEIGHT1 | WEIGHT2 |
| AND  |   -6  |    4    |    4    |
| NOR  |   4  |    -6    |    -6    |

### simplest architecture
![](../images/simplest_xor_architecture.png)

In [113]:
import numpy as np

def sigmoid (x):
	return 1/(1 + np.exp(-x))

def sigmoid_derivative(x):
	return x * (1 - x)

# inputs
inputs = np.array([[0,0],[0,1],[1,0],[1,1]])
expected_output = np.array([[0],[1],[1],[0]])

# hyperparameters
epochs = 10000
lr = 0.1
penalty_coefficient = 0.03
dual_parameters = [4, -6]
threshold = -1

# initialize weigths and biases
inputLayerNeurons, hiddenLayerNeurons, outputLayerNeurons = 2,2,1
hidden_weights = np.random.uniform(low=-6, high=4, size=(inputLayerNeurons,hiddenLayerNeurons))
hidden_bias =np.random.uniform(low=-6, high=4, size=(1,hiddenLayerNeurons))
output_weights = np.random.uniform(low=-6, high=4, size=(hiddenLayerNeurons,outputLayerNeurons))
output_bias = np.random.uniform(low=-6, high=4, size=(1,outputLayerNeurons))

#Training algorithm
for _ in range(epochs):
	#Forward Propagation
	hidden_layer_activation = np.dot(inputs,hidden_weights)
	hidden_layer_activation += hidden_bias
	hidden_layer_output = sigmoid(hidden_layer_activation)

	output_layer_activation = np.dot(hidden_layer_output,output_weights)
	output_layer_activation += output_bias
	predicted_output = sigmoid(output_layer_activation)

	#Backpropagation
	error = expected_output - predicted_output
	d_predicted_output = error * sigmoid_derivative(predicted_output)

	error_hidden_layer = d_predicted_output.dot(output_weights.T)
	d_hidden_layer = error_hidden_layer * sigmoid_derivative(hidden_layer_output)

	#Updating Weights and Biases
	output_weights += hidden_layer_output.T.dot(d_predicted_output) * lr
	output_bias += np.sum(d_predicted_output,axis=0,keepdims=True) * lr
	hidden_weights += inputs.T.dot(d_hidden_layer) * lr
	hidden_bias += np.sum(d_hidden_layer,axis=0,keepdims=True) * lr

	x = 0
	# todo: do this properly by backpropagation with matrix operations
	if(epochs > 20):
		for i, weight in enumerate(output_weights):
			if weight > -1:
				output_weights[i] = weight - 2*penalty_coefficient*(weight-4)
			else:
				output_weights[i] = weight - 2*penalty_coefficient*(weight+6)

		if output_bias[0] > -1:
			output_bias[0] = output_bias[0] - 2*penalty_coefficient*(output_bias[0]-4)
		else:
			output_bias[0] = output_bias[0] - 2*penalty_coefficient*(output_bias[0]+6)

		for i, weights in enumerate(hidden_weights):
			for j, weight in enumerate(weights):
				if weight > -1:
					hidden_weights[i,j] = weight - 2*penalty_coefficient*(weight-4)
				else:
					hidden_weights[i,j] = weight - 2*penalty_coefficient*(weight+6)

		for i, weights in enumerate(hidden_bias):
			for j, weight in enumerate(weights):
				if weight > -1:
					hidden_bias[i,j] = weight - 2*penalty_coefficient*(weight-4)
				else:
					hidden_bias[i,j] = weight - 2*penalty_coefficient*(weight+6)



# results
print("Final hidden weights: ",end='')
print(*hidden_weights)
print("Final hidden bias: ",end='')
print(*hidden_bias)
print("Final output weights: ",end='')
print(*output_weights)
print("Final output bias: ",end='')
print(*output_bias)
print("\nOutput from neural network after 10,000 epochs: ",end='')
print(*predicted_output)

Final hidden weights: [-4.91343871 -1.47922463] [4.00612084 1.58242818]
Final hidden bias: [-2.84768601  1.454252  ]
Final output weights: [4.03872877] [-6.06001498]
Final output bias: [3.95049173]

Output from neural network after 10,000 epochs: [0.323194] [0.77569961] [0.72262805] [0.27666736]
